In [1]:
from agents import Agent, Runner, trace
from dotenv import load_dotenv
from agents.mcp import MCPServerStdio
import os
# Create server parameters for stdio connection
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

from langchain_mcp_adapters.tools import load_mcp_tools 
from langchain.agents import create_agent

In [2]:
load_dotenv(override=True)

True

In [3]:
sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))
files_params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]}

async with MCPServerStdio(params=files_params,client_session_timeout_seconds=60) as server:
    file_tools = await server.list_tools()

file_tools

[Tool(name='read_file', description='Read the complete contents of a file as text. DEPRECATED: Use read_text_file instead.', inputSchema={'$schema': 'http://json-schema.org/draft-07/schema#', 'type': 'object', 'properties': {'path': {'type': 'string'}, 'tail': {'description': 'If provided, returns only the last N lines of the file', 'type': 'number'}, 'head': {'description': 'If provided, returns only the first N lines of the file', 'type': 'number'}}, 'required': ['path']}, annotations=ToolAnnotations(title=None, readOnlyHint=True, destructiveHint=None, idempotentHint=None, openWorldHint=None), title='Read File (Deprecated)', execution={'taskSupport': 'forbidden'}, outputSchema={'$schema': 'http://json-schema.org/draft-07/schema#', 'type': 'object', 'properties': {'content': {'type': 'string'}}, 'required': ['content'], 'additionalProperties': False}),
 Tool(name='read_text_file', description="Read the complete contents of a file from the file system as text. Handles various text enco

In [9]:
sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))

# Make sure sandbox directory exists
os.makedirs(sandbox_path, exist_ok=True)

server_params = StdioServerParameters(
    command="npx",
    args=[
        "-y",
        "@modelcontextprotocol/server-filesystem",
        sandbox_path
    ]
)

async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        await session.initialize()

        tools = await load_mcp_tools(session)
        print("Available tools:")
        for tool in tools:
            print(f"  - {tool.name}")
        
        agent = create_agent("openai:gpt-4o-mini", tools)
        
        # ✅ Write to sandbox/answer.md instead
        agent_response = await agent.ainvoke({
            "messages": "What's (3 + 5) x 12? Write the results to the file named sandbox/answer.md"
        })
        
        print("\nAgent Response:")
        print(agent_response)

Available tools:
  - read_file
  - read_text_file
  - read_media_file
  - read_multiple_files
  - write_file
  - edit_file
  - create_directory
  - list_directory
  - list_directory_with_sizes
  - directory_tree
  - move_file
  - search_files
  - get_file_info
  - list_allowed_directories

Agent Response:
{'messages': [HumanMessage(content="What's (3 + 5) x 12? Write the results to the file named sandbox/answer.md", additional_kwargs={}, response_metadata={}, id='5585bfea-b357-44f7-bb3d-c70a2cd2fc0b'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 1178, 'total_tokens': 1210, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'id': 'chatcmpl